# Transpilazione

In [ ]:
import ast
from pathlib import Path
from textwrap import indent
import webbrowser

from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree

## Un linguaggio semplice

Iniziamo da un linguaggio semplice, con variabili di tipo intero, *scope* esclusivamente globale, e due sole strutture di controllo: la *selezione* e la *ripetizione* (ossia una iterazione controllata in cui il numero di itearti sia noto e fissato prima del suo inizio). Per chi sa cosa sono, questo linguaggio è in grado di esprimere tutte le funzioni *ricorsive primitive*, una conseguenza di questo fatto è che la computazione termina sempre.

Per consentire un embrione di I/O assumeremo che le variabili `INPUT<N>` (dove `<N>` è un intero) e `OUTPUT` siano già pre-dichiarate nell'ambiente d'esecuzione con l'ovvio significato: la prime conterranno l'*input* per il programma e, al termine dell'esecuzione, l'*interprete* emetterà il contenuto della seconda.

### Grammatica e parsing

In [ ]:
SimpleLang = ANTLR(r"""grammar SimpleLang;

program: stat+ ;

stat: 'var' ID ('=' expr)? ';'                    # varDeclStat
    | 'if' expr 'then' stat+ ('else' stat+)? 'fi' # ifElseStat
    | 'repeat' expr 'times' stat+ 'done'          # repeatStat
    | ID '=' expr ';'                             # assignementStat
    ;

expr: '!' expr            # notExpr
    | expr '*' expr       # prodExpr
    | expr ('+'|'-') expr # addSubExpr
    | expr '==' expr      # equalityExpr
    | ID                  # varRefExpr
    | INT                 # intExpr
    | '(' expr ')'        # subExpr
    ;

ID: LETTER (LETTER | [0-9])* ;
INT: [0-9]+ ;
WS: [ \t\n\r]+ -> skip ;
fragment
LETTER : [a-zA-Z] ;
""")

Un esempio di sorgente

In [ ]:
source = """
var somma = 0;
var n = 0;
repeat INPUT0 times
  n = n + 1;
  if n == 3 then
    OUTPUT = 100;
  fi
  if n == 4 then
    OUTPUT = 0;
  else
    OUTPUT = 10;
  fi
  somma = somma + n;
done
OUTPUT = OUTPUT + somma;
"""

L'albero di parsing

In [ ]:
ptree = SimpleLang.tree(source, 'program')
ptree

### Dal parse tree all'AST

In [ ]:
simple2ast = AnnotatedTreeWalker('name')

In [ ]:
@simple2ast.register
def intExpr(visit, ptree):
  return Tree({'type': 'intExpr', 'value': int(ptree.children[0].attr.value)})

@simple2ast.register
def addSubExpr(visit, ptree):
  left, op, right = ptree.children
  return Tree({'type': 'addSubExpr', 'op': op.attr.name}, [visit(left), visit(right)])

@simple2ast.register
def equalityExpr(visit, ptree):
  left, _, right = ptree.children
  return Tree({'type': 'equalityExpr', 'op': '=='}, [visit(left), visit(right)])

@simple2ast.register
def varRefExpr(visit, ptree):
  return Tree({'type': 'varRefExpr', 'varName': ptree.children[0].attr.value})

In [ ]:
@simple2ast.register
def varDeclStat(visit, ptree):
  name = ptree.children[1].attr.value
  if len(ptree.children) == 5:
    return Tree({'type': 'varDeclInitStat', 'varName': name}, [visit(ptree.children[3])])
  else:
    return Tree({'type': 'varDeclStat', 'varName': name})

@simple2ast.register
def assignementStat(visit, ptree):
  name, _, expr, _ = ptree.children
  return Tree({'type': 'assignementStat', 'varName': name.attr.value}, [visit(expr)])

@simple2ast.register
def repeatStat(visit, ptree):
  _, expr, _, *stats, _ = ptree.children
  return Tree({'type': 'repeatStat'}, [visit(expr)] + [visit(stat) for stat in stats])

@simple2ast.register
def ifElseStat(visit, ptree):
  _, cond, _, *stats, _ = ptree.children
  try:
    elsePos = [stat.attr.name for stat in stats].index('else')
  except ValueError:
    elsePos = len(stats)
  return Tree( {'type': 'ifElseStat', 'elsePos': elsePos}, 
               [visit(cond)] +
               [visit(stat) for stat in stats[:elsePos]] +
               [visit(stat) for stat in stats[elsePos + 1:]]
         )

@simple2ast.register
def program(visit, ptree):
  return Tree({'type': 'program'}, [visit(child) for child in ptree.children])

In [ ]:
atree = simple2ast(ptree)
atree

In [ ]:
simple2ast.save('simple2ast.atw') # pre la prossima lezione

### Transpilazione verso Javascript

In [ ]:
simple2js = AnnotatedTreeWalker('type')

In [ ]:
@simple2js.register
def intExpr(visit, atree):
  return str(atree.attr.value)

@simple2js.register
def addSubExpr(visit, atree):
  return f'{visit(atree.children[0])} {atree.attr.op} {visit(atree.children[1])}'

@simple2js.register
def equalityExpr(visit, atree):
  return f'{visit(atree.children[0])} == {visit(atree.children[1])}'

@simple2js.register
def varRefExpr(visit, atree):
  return atree.attr.varName

@simple2js.register
def varDeclStat(visit, atree):
  return f'let {atree.attr.varName};'

@simple2js.register
def varDeclInitStat(visit, atree):
  return f'let {atree.attr.varName} = {visit(atree.children[0])};'

@simple2js.register
def ifElseStat(visit, atree):
  elsePos = atree.attr.elsePos
  stats = atree.children[1:]
  test = visit(atree.children[0])
  body = indent('\n'.join(visit(stat) for stat in stats[:elsePos]), '\t')
  orelse = indent('\n'.join(visit(stat) for stat in stats[elsePos:]), '\t')
  orelse = f'else {{\n{orelse}\n}}' if orelse != '' else ''
  return f'if ({test}) {{\n{body}\n}} {orelse}'

@simple2js.register
def assignementStat(visit, atree):
  return f'{atree.attr.varName} = {visit(atree.children[0])};'

@simple2js.register
def program(visit, atree):
  return '\n'.join(visit(child) for child in atree.children)

In [ ]:
# unica cosa degna di nota, il repeat diventa un for

@simple2js.register
def repeatStat(visit, atree):
  count, *stats = atree.children
  stats = indent('\n'.join(visit(stat) for stat in stats), '\t')
  return f'for (let rv = 0; rv < {visit(count)}; rv++) {{\n{stats}\n}}' 


In [ ]:
code = simple2js(atree)
print(code)

### E l'input/output?

Facendo una analisi successiva al primo passo di transpilazione

In [ ]:
SEEN_INPUT_VARS = set()

def geninput(atree):
  if atree.attr.type == 'varRefExpr':
    name = atree.attr.varName
    if name.startswith('INPUT'): SEEN_INPUT_VARS.add(name)
  else:
    for child in atree.children: geninput(child)

geninput(atree)

INPUT_JS = '\n'.join(f'let {name} = window.prompt("{name}");' for name in SEEN_INPUT_VARS)
print(INPUT_JS)

In [ ]:
code_withio = '\n'.join(['let OUTPUT = 0;', INPUT_JS, code, 'window.alert("OUTPUT " + OUTPUT);'])
print(code_withio)

Oppure svolgendo l'analisi e l'aggiunta del codice durante la transpilazione

In [ ]:
SEEN_INPUT_VARS = set()

@simple2js.register
def varRefExpr(visit, atree):
  name = atree.attr.varName
  if name.startswith('INPUT'): SEEN_INPUT_VARS.add(name)
  return name

@simple2js.register
def program(visit, atree):
  code = '\n'.join(visit(child) for child in atree.children)
  inputs = '\n'.join(f'let {name} = window.prompt("{name}");' for name in SEEN_INPUT_VARS)
  return '\n'.join(['let OUTPUT = 0;', inputs, code, 'window.alert("OUTPUT " + OUTPUT)'])

code = simple2js(atree)
print(code)

### Esecuzione

Per eseguire basta, ad esempio, farne una pagina HTML e aprirla nel browser…

In [ ]:
Path('example.html').write_text(f"""

<html>
  <head>
    <title>Execution</title>
  </head>
  <body>
    <script>{code}</script>
  </body>
</html>

""")

# togliere il commento dalla prossima linea…
# webbrowser.open('example.html');

## Gli AST (nativi) di Python

### Dal sorgente all'AST e sua esplorazione

Si può passare da un *sorgente* all'AST con [parse](https://docs.python.org/3/library/ast.html#ast.parse), qui usiamo `mode = 'eval'` perché il sorgente è una *espressione*.

In [ ]:
pysource = '1 + 3 * 4'

In [ ]:
atree = ast.parse(pysource, mode = 'eval')
atree

In [ ]:
# come stringa

ast.dump(atree)

In [ ]:
# visualizzato come Tree

pyast2tree(atree)

### Compilazione ed esecuzione dell'AST

Un AST può essere compilato con [compile](https://docs.python.org/3/library/functions.html#compile) e quindi, essendo una *espressione*, essere eseguito con [eval](https://docs.python.org/3/library/functions.html#eval).

In [ ]:
# di nuovo 'eval' percché è una espressione

code = compile(atree, filename = '<ast>', mode = 'eval')

In [ ]:
# per valutare si usa eval perché è una espressione
eval(code)

### Costruzione "manuale" di un AST

Per una descrizione dei nodi si veda:

* l'[Abstract Grammar](https://docs.python.org/3/library/ast.html#abstract-grammar) riportata nella documentazione ufficiale e
* la sezione [Meet the Nodes](https://greentreesnakes.readthedocs.io/en/latest/nodes.html) della documentazione "Green Tree Snakes".

In [ ]:
# un esempio: l'espressione tra liste [1, 2]  + ['three']

atree = ast.Expression(
  body = ast.BinOp(
    left = ast.List([ast.Num(1), ast.Num(2)], ctx = ast.Load()),
    op = ast.Add(),
    right = ast.List([ast.Str('three')], ctx = ast.Load())
  )
)

In [ ]:
pyast2tree(atree)

Trattandosi di codice costruito a mano, è necessario sistemare i numeri di linea e colonna con [fix_missing_locations](https://docs.python.org/3/library/ast.html#ast.fix_missing_locations) che decora i nodi dell'AST con le informazioni aggiuntive.

In [ ]:
ast.fix_missing_locations(atree)

In [ ]:
# compilazione ed esecuzione

code = compile(atree, filename = '<ast>', mode = 'eval')
eval(code)

## Transpilazione verso gli AST (nativi) di Python

### Costruzione dell'AST Python

In [ ]:
simple2py = AnnotatedTreeWalker('type')

In [ ]:
@simple2py.register
def intExpr(visit, atree):
  return ast.Num(n = atree.attr.value)

@simple2py.register
def addSubExpr(visit, atree):
  left = visit(atree.children[0])
  right = visit(atree.children[1])
  return ast.BinOp(
    left = left, 
    op = ast.Add() if atree.attr.op == '+' else ast.Sub(), 
    right = right
  )

@simple2py.register
def equalityExpr(visit, atree):
  left = visit(atree.children[0])
  right = visit(atree.children[1])
  return ast.Compare(left = left, ops=[ast.Eq()], comparators = [right])

@simple2py.register
def varRefExpr(visit, atree):
  return ast.Name(id = atree.attr.varName, ctx = ast.Load())

@simple2py.register
def varDeclStat(visit, atree):
  return ast.Assign(targets = [ast.Name(id = atree.attr.varName, ctx = ast.Store())], value = ast.Num(n = 0))

@simple2py.register
def varDeclInitStat(visit, atree):
  return ast.Assign(targets = [ast.Name(id = atree.attr.varName, ctx = ast.Store())], value = visit(atree.children[0]))

@simple2py.register
def ifElseStat(visit, atree):
  elsePos = atree.attr.elsePos
  stats = atree.children[1:]
  return ast.If(
    test = visit(atree.children[0]), 
    body = [visit(stat) for stat in stats[:elsePos]], 
    orelse = [visit(stat) for stat in stats[elsePos:]], 
  )

@simple2py.register
def assignementStat(visit, atree):
  return ast.Assign(targets = [ast.Name(id = atree.attr.varName, ctx = ast.Store())], value = visit(atree.children[0]))

@simple2py.register
def program(visit, atree):
  return ast.Module(body = [visit(child) for child in atree.children], type_ignores = [])

In [ ]:
# di nuovo, unica cosa degna di nota, il repeat diventa un for che usa la funzione range

@simple2py.register
def repeatStat(visit, atree):
  count, *stats = atree.children
  return ast.For(
    target = ast.Name(id = 'rv', ctx = ast.Store()), 
    iter = ast.Call( 
             func = ast.Name(id = 'range', ctx = ast.Load()), 
             args = [visit(count)], 
             keywords = []
           ), 
    body = [visit(stat) for stat in stats], 
    orelse = []
  )

In [ ]:
# costruzione dell'AST Python 

pyast = simple2py(simple2ast(SimpleLang.tree(source, 'program')))

In [ ]:
pyast2tree(pyast)

### Esecuzione

Per prima cosa, si sistemano i numeri di linea e si compila, in questo caso con `mode = 'exec'` perché il codice è uno *statement* non più una espressione.

In [ ]:
ast.fix_missing_locations(pyast)

In [ ]:
code = compile(pyast, filename = '<ast>', mode = 'exec')

Prima dell'esecuzione con [exec](https://docs.python.org/3/library/functions.html#exec) è necessario fornire il dizionario delle variabili locali, dove mettere l'*input*:

In [ ]:
localvars = {'INPUT0': 10}

In [ ]:
# esecuzione

exec(code, globals(), localvars)

In [ ]:
# reperimento dell'output

localvars['OUTPUT']